# calling libraries

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
import os
import random
import tensorflow.keras as keras
# from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import mean_absolute_percentage_error
import pywt

import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.signal import butter, lfilter
from sklearn.metrics import mean_squared_error

In [ ]:
INPUT_SIZE = 8

# main functions

In [ ]:
#This is for getting the same results.
def reset_random_seeds(seed):
   os.environ['PYTHONHASHSEED']=str(seed)
   tf.random.set_seed(seed)
   np.random.seed(seed)
   random.seed(seed)
reset_random_seeds(42)

def successive(successive):
    input_data=[]
    for i in range(2 * (INPUT_SIZE-1), len(successive)):
        if INPUT_SIZE == 4:
            input_data.append([successive[i-3]]+[successive[i-2]]+[successive[i-1]]+[successive[i]])
        else:
            input_data.append([successive[i-7]]+[successive[i-6]]+[successive[i-5]]+[successive[i-4]]+[successive[i-3]]+[successive[i-2]]+[successive[i-1]]+[successive[i]])
    return input_data  

#wavelet transform
def dwt(training):
    input_data=np.array(training)
    days = input_data[:,0:INPUT_SIZE]
    (a, d) = pywt.dwt(days, 'haar')
    (a2,d2)=pywt.dwt(a, 'haar')
    if INPUT_SIZE == 4:
        l3=np.append(a2,d2, axis=1)
        l2_3=np.append(l3,d, axis=1)
        transformed_df=l2_3        
    else:
        (a3,d3)=pywt.dwt(a2, 'haar')
        l3=np.append(a3,d3, axis=1)
        l2_3=np.append(l3,d2, axis=1)
        l3_3=np.append(l2_3,d, axis=1)
        transformed_df=l3_3     

    training=transformed_df[:,1:]
    return training

def idwt(transformed_data):
    (a, d) = np.hsplit(transformed_data, 2)
    (a2, d2) = np.hsplit(a, 2)
    if INPUT_SIZE == 4:
        idwt_res = pywt.idwt(a2, d2, 'haar')
        reconstructed_df = pywt.idwt(idwt_res, d, 'haar')
    else:
        (a3, d3) = np.hsplit(a2, 2)
        idwt_res = pywt.idwt(a3, d3, 'haar')
        idwt_res2 = pywt.idwt(idwt_res, d2, 'haar')
        reconstructed_df = pywt.idwt(idwt_res2, d, 'haar')
    
    return reconstructed_df

def get_data(path, type, start, stop):
    input_data = pd.read_csv(path)
    input_data = np.array(input_data)[start:stop][:,type]
    input_data = np.array(input_data)
    input_data = input_data.reshape(input_data.shape[0])
    input_data = list(input_data)
    
    return input_data

def preprocess_data(data_list):
    if len(data_list) == 1:
        input_data = data_list[0]
        shifted_input_data = input_data[24-INPUT_SIZE:]
        input_data = np.array(input_data[:-24])
        out_data = shifted_input_data[INPUT_SIZE:]
        shifted_input_data = np.array(shifted_input_data[:-INPUT_SIZE])
        input_data_successive = successive(input_data)
        second_data_successive = successive(shifted_input_data)
        out_data_successive = successive(out_data)
    else:
        input_data = np.array(data_list[0])
        shifted_input_data = np.array(data_list[1])
        out_data = np.array(data_list[2])
        input_data_successive = successive(input_data)
        second_data_successive = successive(shifted_input_data)
        out_data_successive = successive(out_data)
    return input_data, out_data_successive, input_data_successive, second_data_successive

In [ ]:
#network configurations
hidden1=16
second_layer1=16
third_layer1=16
forth_layer1=8

hidden2=16
second_layer2=16
third_layer2=16
forth_layer2=8

hidden3=16
second_layer3=16
third_layer3=16
forth_layer3=8

hidden4=16
second_layer4=16
third_layer4=16
forth_layer4=8

hidden5=16
second_layer5=16
third_layer5=16
forth_layer5=8

## Preprocessing

In [ ]:
data_list = []
data_list.append(get_data('original_code/data/hourly/63.11.HG.csv', type=5, start=12077, stop=13346))
data_list.append(get_data('original_code/data/hourly/63.13.HG.csv', type=9, start=11977, stop=13246))
data_list.append(get_data('original_code/data/hourly/63.12.HG.csv', type=13, start=12065, stop=13334))
input_data, out_data_successive, input_data_successive, second_data_successive = preprocess_data(data_list)
# type
# 5: Temperature
# 9: Relative Humidity
# 13: Wind Speed

#division of data set into training and test data set
N = len(input_data)
division_of_training = 0.9
input_train = input_data_successive[:int(N*division_of_training)]
input_test = input_data_successive[int(N*division_of_training):int(N)]

input_train_today = second_data_successive[:int(N*division_of_training)]
input_test_today = second_data_successive[int(N*division_of_training):int(N)]

output_train = out_data_successive[:int(N*division_of_training)]
output_test = out_data_successive[int(N*division_of_training):int(N)]

#normalization

subtraction_first_train = np.mean(np.array(input_train), axis=1)
subtraction_first_test = np.mean(np.array(input_test), axis=1)

subtraction_second_train = np.mean(np.array(input_train_today), axis=1)
subtraction_second_test = np.mean(np.array(input_test_today), axis=1)

subtraction_output_train = np.mean(np.array(output_train), axis=1)
subtraction_output_test = np.mean(np.array(output_test), axis=1)

#normalization of inputs

first_input_train=input_train-subtraction_first_train[:, None]
first_input_test=input_test-subtraction_first_test[:, None]

output_train_normalized=output_train-subtraction_output_train[:, None]
output_test_normalized=output_test-subtraction_output_test[:, None]

second_input_train=input_train_today-subtraction_second_train[:, None]
second_input_test=input_test_today-subtraction_second_test[:, None]

#4inputs WT
final_first_w_input_train=dwt(first_input_train)
final_first_w_input_test=dwt(first_input_test)
output_w_train = dwt(output_train_normalized)
output_w_test = dwt(output_test_normalized)

## PECNET function

In [ ]:
def PECNET(X_train, y_train, X_test, y_test):
    X_train=np.array(X_train)
    y_train=np.array(y_train)

    X_test=np.array(X_test)
    y_test=np.array(y_test)

    m_primary=len(X_train[0,:])
    p_primary=np.size(y_train[0])
    N_primary=len(X_train)

    model= Sequential ([
        Dense(hidden1, input_dim=m_primary, activation='relu'), 
        Dropout(0.1),
        Dense(second_layer1), #,activation='relu'),
        Dropout(0.1),
        Dense(third_layer1), #,activation='relu'),
        Dropout(0.1),
        Dense(forth_layer1), #,activation='relu'),
        Dropout(0.1),
        Dense(p_primary)
        ])
        
    #model.summary()

    #sgd=keras.optimizers.legacy.SGD(lr=0.05,momentum=0.75, decay=0.0, nesterov=False)
    rmsprop = keras.optimizers.legacy.RMSprop(lr=0.05,momentum=0.75, decay=0.0)
    adam = keras.optimizers.legacy.Adam(learning_rate=0.01)
    model.compile(loss='mean_squared_error', optimizer=adam, metrics=['mean_absolute_error','mean_squared_logarithmic_error','cosine_similarity','logcosh'])
    history1=model.fit(X_train, y_train, batch_size=int(N_primary/10), epochs=300, shuffle=False, verbose=0)  

    predicted_train = model.predict(X_train) 
    predicted_train = np.reshape(predicted_train, (predicted_train.size,))
    error_train1=predicted_train-y_train

    predicted_test = model.predict(X_test) 
    predicted_test = np.reshape(predicted_test, (predicted_test.size,))
    error_test1=predicted_test-y_test

    error_train=pd.DataFrame(error_train1)
    add_train=dwt(second_input_train) 
    
    X_error_train1=np.array(add_train)
    y_error_train1=np.array(error_train)

    error_test=pd.DataFrame(error_test1)
    add_test=dwt(second_input_test) 

    X_error_test1=np.array(add_test)

    m_second=len(X_error_train1[0,:])
    p_second=np.size(y_train[0])
    N_second=len(X_error_train1)

    error_model1= Sequential ([
        Dense(hidden2, input_dim=m_second, activation='relu'), 
        Dropout(0.1),
        Dense(second_layer2), #,activation='relu'),
        Dropout(0.1),
        Dense(third_layer2), #,activation='relu'),
        Dropout(0.1),
        Dense(forth_layer2), #,activation='relu'),
        Dropout(0.1),
        Dense(p_second)
    ])

    #error_model1.summary()

    #sgd=keras.optimizers.legacy.SGD(learning_rate=0.05, momentum=0.75, decay=0.0, nesterov=False)
    rmsprop = keras.optimizers.legacy.RMSprop(lr=0.05,momentum=0.75, decay=0.0)
    error_model1.compile(loss='mean_squared_error', optimizer=adam, metrics=['mse','mae','accuracy'])
    history3=error_model1.fit(X_error_train1, y_error_train1, batch_size=N_second, epochs=300, shuffle=False, verbose=0)

    error_predicted_tr = error_model1.predict(X_error_train1)
    error_predicted_tr = np.reshape(error_predicted_tr, (error_predicted_tr.size,))
    error_predicted_tes = error_model1.predict(X_error_test1)
    error_predicted_tes = np.reshape(error_predicted_tes, (error_predicted_tes.size,))

    compensated1_train=(predicted_train+subtraction_second_train)-(error_predicted_tr)
    compensated1_test=(predicted_test+subtraction_second_test)-(error_predicted_tes)

    error_train2a=compensated1_train-(y_train+subtraction_second_train)
    error_test2a=compensated1_test-(y_test+subtraction_second_test)

    error_train2=pd.DataFrame(error_train2a)
    for i in range(INPUT_SIZE):
        error_train2[i+1]= error_train2[i].shift(1)
    error_train2 = error_train2.replace(np.nan, 0)
    error_train2 = error_train2.iloc[:,::-1]
    ##error normalization
    subtraction_error_train2=np.mean(np.array(error_train2)[:,:-1], axis=1)
    error_train2=error_train2-subtraction_error_train2[:, None]
    error_train2=np.array(error_train2)
    days_train = error_train2[:,:INPUT_SIZE]
    input3_train=dwt(days_train)
    output3_train=error_train2[:,INPUT_SIZE]

    X_error_train2=np.array(input3_train)
    y_error_train2=np.array(output3_train)

    error_test2=pd.DataFrame(error_test2a)
    for i in range(INPUT_SIZE):
        error_test2[i+1]= error_test2[i].shift(1)
    error_test2 = error_test2.replace(np.nan, 0)
    error_test2 = error_test2.iloc[:,::-1]

    subtraction_error_test2=np.array(error_test2)
    subtraction_error_test2=subtraction_error_test2[:,:-1]
    subtraction_error_test2=np.mean(subtraction_error_test2, axis=1)
    error_test2=error_test2-subtraction_error_test2[:, None]

    error_test2=np.array(error_test2)
    days_test = error_test2[:,:INPUT_SIZE]
    input3_test=dwt(days_test)
    output3_test=error_test2[:,INPUT_SIZE]

    X_error_test2=np.array(input3_test)

    #####3rd NN
    m_error=len(X_error_train2[0,:])
    p_error=np.size(y_error_train2[0])
    N_error=len(X_error_train2)

    error_model2= Sequential ([
        Dense(hidden3, input_dim=m_error, activation='relu'), 
        Dropout(0.1),
        Dense(second_layer3), #,activation='relu'),
        Dropout(0.1),
        Dense(third_layer3), #,activation='relu'),
        Dropout(0.1),
        Dense(forth_layer3), #,activation='relu'),
        Dropout(0.1),
        Dense(p_error)
    ])

    #error_model2.summary()

    #sgd=keras.optimizers.legacy.SGD(learning_rate=0.05, momentum=0.75, decay=0.0, nesterov=False)
    rmsprop = keras.optimizers.legacy.RMSprop(lr=0.05,momentum=0.75, decay=0.0)
    error_model2.compile(loss='mean_squared_error', optimizer=adam, metrics=['mse','mae','accuracy'])
    history4=error_model2.fit(X_error_train2, y_error_train2, batch_size=N_error, epochs=300, shuffle=False, verbose=0)


    error_predicted_tr2 = error_model2.predict(X_error_train2)
    error_predicted_tr2 = np.reshape(error_predicted_tr2, (error_predicted_tr2.size,))
    error_predicted_tes2 = error_model2.predict( X_error_test2)
    error_predicted_tes2= np.reshape(error_predicted_tes2, (error_predicted_tes2.size,))

    compensated_y_train=compensated1_train-(error_predicted_tr2+subtraction_error_train2)
    compensated_y_test=compensated1_test-(error_predicted_tes2+subtraction_error_test2)

    error_predicted_tr3=error_predicted_tr2+subtraction_error_train2
    error_predicted_tes3=error_predicted_tes2+subtraction_error_test2

    training_final_add=np.column_stack((predicted_train, error_predicted_tr))
    training_final_add=np.column_stack((training_final_add,error_predicted_tr3))

    test_final_add=np.column_stack((predicted_test, error_predicted_tes))
    test_final_add=np.column_stack((test_final_add,error_predicted_tes3))

    ####final NN
    m_final=len(training_final_add[0,:])
    p_final=np.size(y_train[0])
    N_final=len(training_final_add)

    final_model= Sequential ([
        Dense(hidden4, input_dim=m_final, activation='relu'), 
    #    Dropout(0.1),
    #    Dense(second_layer4), #,activation='relu'),
    #    Dropout(0.1),
    #    Dense(third_layer4), #,activation='relu'),
    #    Dropout(0.1),
    #    Dense(forth_layer4), #,activation='relu'),
    #    Dropout(0.1),
        Dense(p_final)
    ])

    #final_model.summary()

    #sgd=keras.optimizers.legacy.SGD(learning_rate=0.05, momentum=0.75, decay=0.0, nesterov=False)
    rmsprop = keras.optimizers.legacy.RMSprop(lr=0.05,momentum=0.75, decay=0.0)
    final_model.compile(loss='mean_squared_error', optimizer=adam, metrics=['mse','mae','accuracy'])
    final_history=final_model.fit(training_final_add, y_train, batch_size=N_final, epochs=300, shuffle=False, verbose=0)

    final_predicted_tr = final_model.predict(training_final_add)
    final_predicted_tr = np.reshape(final_predicted_tr, (final_predicted_tr.size,))
    final_predicted_tes = final_model.predict(test_final_add)
    final_predicted_tes = np.reshape(final_predicted_tes, (final_predicted_tes.size,))

    y_train=y_train+subtraction_second_train
    final_y_train=final_predicted_tr+subtraction_second_train
    final_y_train = np.reshape(final_y_train, (final_y_train.size,))

    final_error_train=final_y_train-y_train
    final_rmse_error_train=np.sqrt(sum(final_error_train*final_error_train)/len(final_error_train))
    final_mse_train=(sum(final_error_train*final_error_train)/len(final_error_train))
    final_mape_train=100*sum(abs(final_error_train/y_train))/len(y_train)
    final_mae_train=sum(abs(final_error_train-y_train))/len(y_train)
    final_rmspe_train=100*np.sqrt(np.nanmean(np.square(((y_train - final_y_train) / y_train))))

    
    y_test=y_test+subtraction_second_test
    
    final_y_test=final_predicted_tes+subtraction_second_test
    y_test = np.reshape(y_test, (y_test.size,))
    final_y_test = np.reshape(final_y_test, (final_y_test.size,))


    #final_error_test=y_test[:-1]-final_predicted_tes[:-1]
    final_error_test=final_y_test[:-1]-y_test[:-1] 
    final_rmse_error_test=np.sqrt(sum(final_error_test*final_error_test)/len(final_error_test))
    final_mse_test=(sum(final_error_test*final_error_test)/len(final_error_test))
    final_mape_test=100*sum(abs(final_error_test/y_test[:-1]))/len(y_test-1)
    final_mae_test=sum(abs(final_error_test-y_test[:-1]))/len(y_test-1)
    final_rmspe_test=100*np.sqrt(np.nanmean(np.square(((y_test[:-1] - final_y_test[:-1]) / y_test[:-1]))))

    #errors of the first nn
    predicted_train=predicted_train+subtraction_second_train
    predicted_test=predicted_test+subtraction_second_test

    predicted_error_train=predicted_train-y_train
    predicted_rmse_error_train=np.sqrt(sum(predicted_error_train*predicted_error_train)/len(predicted_error_train))
    predicted_mse_train=(sum(predicted_error_train*predicted_error_train)/len(predicted_error_train))
    predicted_mape_train=100*sum(abs(predicted_error_train/y_train))/len(y_train)
    predicted_mae_train=sum(abs(predicted_error_train-y_train))/len(y_train)
    predicted_rmspe_train=100*np.sqrt(np.nanmean(np.square(((y_train - predicted_train) / y_train))))

    predicted_error_test=predicted_test[:-1]-y_test[:-1]
    predicted_rmse_error_test=np.sqrt(sum(predicted_error_test*predicted_error_test)/len(predicted_error_test))
    predicted_mse_test=(sum(predicted_error_test*predicted_error_test)/len(predicted_error_test))
    predicted_mape_test=100*sum(abs(predicted_error_test/y_test[:-1]))/len(y_test-1)
    predicted_mae_test=sum(abs(predicted_error_test-y_test[:-1]))/len(y_test-1)
    predicted_rmspe_test=100*np.sqrt(np.nanmean(np.square(((y_test[:-1] - predicted_test[:-1]) / y_test[:-1]))))

    #errors of the second nn
    compensated1_train_error=compensated1_train-y_train

    compensated1_train_rmse_error_train=np.sqrt(sum(compensated1_train_error*compensated1_train_error)/len(compensated1_train_error))
    compensated1_train_mse_train=(sum(compensated1_train_error*compensated1_train_error)/len(compensated1_train_error))
    compensated1_train_mape_train=100*sum(abs(compensated1_train_error/y_train))/len(y_train)
    compensated1_train_mae_train=sum(abs(compensated1_train_error-y_train))/len(y_train)
    compensated1_train_rmspe_train=np.sqrt(np.nanmean(np.square(((y_train - compensated1_train) / y_train))))*100

    compensated1_test_error=compensated1_test[:-1]-y_test[:-1]

    compensated1_test_rmse_error_test=np.sqrt(sum(compensated1_test_error*compensated1_test_error)/len(compensated1_test_error))
    compensated1_test_mse_test=(sum(compensated1_test_error*compensated1_test_error)/len(compensated1_test_error))
    compensated1_test_mape_test=100*sum(abs(compensated1_test_error/y_test[:-1]))/len(y_test-1)
    compensated1_test_mae_test=sum(abs(compensated1_test_error-y_test[:-1]))/len(y_test-1)
    compensated1_test_rmspe_test=np.sqrt(np.nanmean(np.square(((y_test[:-1] - compensated1_test[:-1]) / y_test[:-1]))))*100

    #errors of the third nn
    compensated_error_train=compensated_y_train-y_train

    comp_rmse_error_train=np.sqrt(sum(compensated_error_train*compensated_error_train)/len(compensated_error_train))
    comp_mse_train=(sum(compensated_error_train*compensated_error_train)/len(compensated_error_train))
    comp_mape_train=100*sum(abs(compensated_error_train/y_train))/len(y_train)
    comp_mae_train=sum(abs(compensated_error_train-y_train))/len(y_train)
    comp_rmspe_train=np.sqrt(np.nanmean(np.square(((y_train - compensated_y_train) / y_train))))*100

    compensated_error_test=compensated_y_test[:-1]-y_test[:-1]

    comp_rmse_error_test=np.sqrt(sum(compensated_error_test*compensated_error_test)/len(compensated_error_test))
    comp_mse_test=(sum(compensated_error_test*compensated_error_test)/len(compensated_error_test))
    comp_mape_test=100*sum(abs(compensated_error_test/y_test[:-1]))/len(y_test-1)
    comp_mae_test=sum(abs(compensated_error_test-y_test[:-1]))/len(y_test-1)
    comp_rmspe_test=np.sqrt(np.nanmean(np.square(((y_test[:-1] - compensated_y_test[:-1]) / y_test[:-1]))))*100

    zz_rmse_errors_ttrain=(predicted_rmse_error_train,compensated1_train_rmse_error_train, comp_rmse_error_train,final_rmse_error_train)
    zz_rmse_errors_test=(predicted_rmse_error_test,compensated1_test_rmse_error_test, comp_rmse_error_test,final_rmse_error_test)

    zz_rmspe_errors_ttrain=(predicted_rmspe_train,compensated1_train_rmspe_train, comp_rmspe_train,final_rmspe_train)
    zz_rmspe_errors_test=(predicted_rmspe_test,compensated1_test_rmspe_test, comp_rmspe_test,final_rmspe_test)

    zz_mape_errors_ttrain=(predicted_mape_train,compensated1_train_mape_train, comp_mape_train,final_mape_train)
    zz_mape_errors_test=(predicted_mape_test,compensated1_test_mape_test, comp_mape_test,final_mape_test)

    zz_mae_errors_ttrain=(predicted_mae_train,compensated1_train_mae_train, comp_mae_train,final_mae_train)
    zz_mae_errors_test=(predicted_mae_test,compensated1_test_mae_test, comp_mae_test,final_mae_test)

    zz_predictions_train = (y_train, predicted_train,compensated1_train,  compensated_y_train, final_y_train)
    zz_predictions_test = (y_test,predicted_test,compensated1_test, compensated_y_test, final_y_test)

    return final_predicted_tes

## Finding predictions

In [ ]:
# WOUT DWT

preds = []
for i in range(INPUT_SIZE):
    preds.append(PECNET(final_first_w_input_train, output_w_train[:,i], final_first_w_input_test, output_w_test[:,i]))
preds = np.array(preds)
preds_last = []
for i in range(len(output_test)):
    preds_last.append(np.array(preds[:,i]))
preds_last = np.array(preds_last)

predicted_data_pecnet_wout_dwt = preds_last + subtraction_output_test[:, None]

In [ ]:
# WITH DWT

preds = []
for i in range(INPUT_SIZE-1):
    preds.append(PECNET(final_first_w_input_train, output_w_train[:,i], final_first_w_input_test, output_w_test[:,i]))
preds = np.array(preds)
preds_w = []
for i in range(len(output_test)):
    w_coeffs = []
    w_coeffs.append(0)
    for elm in preds[:,i]:
        w_coeffs.append(elm)
    preds_w.append(np.array(w_coeffs))
preds_w = np.array(preds_w)

predicted_data_pecnet_dwt = idwt(preds_w) + subtraction_output_test[:, None]

In [ ]:
fig, ((ax0,ax1),(ax2,ax3)) = plt.subplots(2,2, figsize=(10,10))
plt.suptitle('Temperature Prediction')
ax0.plot(history1.history["loss"])
ax0.set_title("First Network (X=Average)")
ax0.set_xlabel("Epoch")
ax0.set_ylabel("Loss")

ax1.plot(history3.history["loss"])
ax1.set_title("Second Network (X=Raw)")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")

ax2.plot(history4.history["loss"])
ax2.set_title("Error Network")
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Loss")

ax3.plot(final_history.history["loss"])
ax3.set_title("Final Network")
ax3.set_xlabel("Epoch")
ax3.set_ylabel("Loss")

# errors

In [ ]:
y_train=y_train+subtraction_second_train
final_y_train=final_predicted_tr+subtraction_second_train
final_y_train = np.reshape(final_y_train, (final_y_train.size,))

final_error_train=final_y_train-y_train
final_rmse_error_train=np.sqrt(sum(final_error_train*final_error_train)/len(final_error_train))
final_mse_train=(sum(final_error_train*final_error_train)/len(final_error_train))
final_mape_train=100*sum(abs(final_error_train/y_train))/len(y_train)
final_mae_train=sum(abs(final_error_train-y_train))/len(y_train)
final_rmspe_train=100*np.sqrt(np.nanmean(np.square(((y_train - final_y_train) / y_train))))

 
y_test=y_test+subtraction_second_test
 
final_y_test=final_predicted_tes+subtraction_second_test
y_test = np.reshape(y_test, (y_test.size,))
final_y_test = np.reshape(final_y_test, (final_y_test.size,))


#final_error_test=y_test[:-1]-final_predicted_tes[:-1]
final_error_test=final_y_test[:-1]-y_test[:-1] 
final_rmse_error_test=np.sqrt(sum(final_error_test*final_error_test)/len(final_error_test))
final_mse_test=(sum(final_error_test*final_error_test)/len(final_error_test))
final_mape_test=100*sum(abs(final_error_test/y_test[:-1]))/len(y_test-1)
final_mae_test=sum(abs(final_error_test-y_test[:-1]))/len(y_test-1)
final_rmspe_test=100*np.sqrt(np.nanmean(np.square(((y_test[:-1] - final_y_test[:-1]) / y_test[:-1]))))

#errors of the first nn
predicted_train=predicted_train+subtraction_second_train
predicted_test=predicted_test+subtraction_second_test

predicted_error_train=predicted_train-y_train
predicted_rmse_error_train=np.sqrt(sum(predicted_error_train*predicted_error_train)/len(predicted_error_train))
predicted_mse_train=(sum(predicted_error_train*predicted_error_train)/len(predicted_error_train))
predicted_mape_train=100*sum(abs(predicted_error_train/y_train))/len(y_train)
predicted_mae_train=sum(abs(predicted_error_train-y_train))/len(y_train)
predicted_rmspe_train=100*np.sqrt(np.nanmean(np.square(((y_train - predicted_train) / y_train))))

predicted_error_test=predicted_test[:-1]-y_test[:-1]
predicted_rmse_error_test=np.sqrt(sum(predicted_error_test*predicted_error_test)/len(predicted_error_test))
predicted_mse_test=(sum(predicted_error_test*predicted_error_test)/len(predicted_error_test))
predicted_mape_test=100*sum(abs(predicted_error_test/y_test[:-1]))/len(y_test-1)
predicted_mae_test=sum(abs(predicted_error_test-y_test[:-1]))/len(y_test-1)
predicted_rmspe_test=100*np.sqrt(np.nanmean(np.square(((y_test[:-1] - predicted_test[:-1]) / y_test[:-1]))))

#errors of the second nn
compensated1_train_error=compensated1_train-y_train

compensated1_train_rmse_error_train=np.sqrt(sum(compensated1_train_error*compensated1_train_error)/len(compensated1_train_error))
compensated1_train_mse_train=(sum(compensated1_train_error*compensated1_train_error)/len(compensated1_train_error))
compensated1_train_mape_train=100*sum(abs(compensated1_train_error/y_train))/len(y_train)
compensated1_train_mae_train=sum(abs(compensated1_train_error-y_train))/len(y_train)
compensated1_train_rmspe_train=np.sqrt(np.nanmean(np.square(((y_train - compensated1_train) / y_train))))*100

compensated1_test_error=compensated1_test[:-1]-y_test[:-1]

compensated1_test_rmse_error_test=np.sqrt(sum(compensated1_test_error*compensated1_test_error)/len(compensated1_test_error))
compensated1_test_mse_test=(sum(compensated1_test_error*compensated1_test_error)/len(compensated1_test_error))
compensated1_test_mape_test=100*sum(abs(compensated1_test_error/y_test[:-1]))/len(y_test-1)
compensated1_test_mae_test=sum(abs(compensated1_test_error-y_test[:-1]))/len(y_test-1)
compensated1_test_rmspe_test=np.sqrt(np.nanmean(np.square(((y_test[:-1] - compensated1_test[:-1]) / y_test[:-1]))))*100

#errors of the third nn
compensated_error_train=compensated_y_train-y_train

comp_rmse_error_train=np.sqrt(sum(compensated_error_train*compensated_error_train)/len(compensated_error_train))
comp_mse_train=(sum(compensated_error_train*compensated_error_train)/len(compensated_error_train))
comp_mape_train=100*sum(abs(compensated_error_train/y_train))/len(y_train)
comp_mae_train=sum(abs(compensated_error_train-y_train))/len(y_train)
comp_rmspe_train=np.sqrt(np.nanmean(np.square(((y_train - compensated_y_train) / y_train))))*100

compensated_error_test=compensated_y_test[:-1]-y_test[:-1]

comp_rmse_error_test=np.sqrt(sum(compensated_error_test*compensated_error_test)/len(compensated_error_test))
comp_mse_test=(sum(compensated_error_test*compensated_error_test)/len(compensated_error_test))
comp_mape_test=100*sum(abs(compensated_error_test/y_test[:-1]))/len(y_test-1)
comp_mae_test=sum(abs(compensated_error_test-y_test[:-1]))/len(y_test-1)
comp_rmspe_test=np.sqrt(np.nanmean(np.square(((y_test[:-1] - compensated_y_test[:-1]) / y_test[:-1]))))*100


zz_rmse_errors_ttrain=(predicted_rmse_error_train,compensated1_train_rmse_error_train, comp_rmse_error_train,final_rmse_error_train)
zz_rmse_errors_test=(predicted_rmse_error_test,compensated1_test_rmse_error_test, comp_rmse_error_test,final_rmse_error_test)

zz_rmspe_errors_ttrain=(predicted_rmspe_train,compensated1_train_rmspe_train, comp_rmspe_train,final_rmspe_train)
zz_rmspe_errors_test=(predicted_rmspe_test,compensated1_test_rmspe_test, comp_rmspe_test,final_rmspe_test)

zz_mape_errors_ttrain=(predicted_mape_train,compensated1_train_mape_train, comp_mape_train,final_mape_train)
zz_mape_errors_test=(predicted_mape_test,compensated1_test_mape_test, comp_mape_test,final_mape_test)

zz_mae_errors_ttrain=(predicted_mae_train,compensated1_train_mae_train, comp_mae_train,final_mae_train)
zz_mae_errors_test=(predicted_mae_test,compensated1_test_mae_test, comp_mae_test,final_mae_test)

zz_predictions_train = (y_train, predicted_train,compensated1_train,  compensated_y_train, final_y_train)
zz_predictions_test = (y_test,predicted_test,compensated1_test, compensated_y_test, final_y_test)

In [ ]:
print("PECNET with wavelet MAPE:", mean_squared_error(get_data('original_code/data/hourly/63.11.HG.csv', type=13, start=12077, stop=13346), get_data('original_code/data/hourly/63.12.HG.csv', type=13, start=12065, stop=13334)))

## Other Methods

In [ ]:
### ONE SHIFTED PREDICTION ###################################################################################

y_shifted = np.array(output_test[1:])
print("One Shifted Input RMSE:", mean_squared_error(output_test[:-1], y_shifted, squared=False))
actual_data = [data for data in np.array(output_test)[:,0]][:-1][-100:]
predicted_data = [data for data in y_shifted[:,0]][-100:]

plt.plot(range(len(actual_data)), actual_data, label='Actual (Test)')
plt.plot(range(len(actual_data)), predicted_data, label='Predicted (Test)')
plt.xlabel('Time')
plt.ylabel(data_type)
plt.title('One Shifted Input (Window Size %d)' % INPUT_SIZE)
plt.legend()
plt.show()

### STATISTICAL PREDICTION ###################################################################################

stat_preds = []
actual_values = []
for i in range(len(output_test)):
    data = output_test
    if i-24-INPUT_SIZE < 0:
        data = output_train
    yesterday = data[i-24-INPUT_SIZE]
    data = output_test
    if i-24 < 0:
        data = output_train
    next_yesterday = data[i-24]
    data = output_test
    if i-INPUT_SIZE < 0:
        data = output_train
    today = data[i-INPUT_SIZE]
    data = output_test
    mean_yesterday = np.mean(yesterday)
    std_yesterday = np.std(yesterday)
    mean_today = np.mean(today)
    std_today = np.std(today)

    mean_proportion = mean_today/mean_yesterday
    std_proportion = std_today/std_yesterday
    pred = [data * mean_proportion for data in next_yesterday]
    stat_preds.append(pred)

    actual_values.append(output_test[i])

print("Statistical MAPE:", mean_absolute_percentage_error(stat_preds, actual_values))

actual_data = [data for data in np.array(actual_values)[:,0]][-100:]
predicted_data = [data for data in np.array(stat_preds)[:,0]][-100:]

plt.plot(range(len(actual_data)), actual_data, label='Actual (Test)')
plt.plot(range(len(actual_data)), predicted_data, label='Predicted (Test)')
plt.xlabel('Time')
plt.ylabel(data_type)
plt.title('Statistical Approach (Window Size %d)' % INPUT_SIZE)
plt.legend()
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

### LINEAR REGRESSION ###################################################################################

merged_successive_train = []
for i in range(len(input_train)):
    merged_successive_train.append(input_train[i] + input_train_today[i])

merged_successive_test = []
for i in range(len(input_test)):
    merged_successive_test.append(input_test[i] + input_test_today[i])

# Initialize linear regression model
model = LinearRegression()

# Train the model
model.fit(merged_successive_train, output_train)

# Predict the next data point iteratively
predicted_data_lr = model.predict(merged_successive_test)

### MLP ###################################################################################

# Initialize MLP regressor model
model = MLPRegressor(hidden_layer_sizes=(100, 100), random_state=42)

# Train the model
model.fit(merged_successive_train, output_train)

# Predict the next data point iteratively
predicted_data_mlp = model.predict(merged_successive_test)

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

input_data=pd.read_csv('original_code/data/hourly/63.12.HG.csv')
input_data=np.array(input_data)[12065:13334][:,5]
# Example input data
temperature_data = list(input_data)

# Set the number of previous hours to use for prediction
look_back = 2*INPUT_SIZE

# Preprocess the data
def create_dataset(data, look_back):
    X, Y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i + look_back])
        Y.append(data[i + look_back])
    return np.array(X), np.array(Y)

X, Y = create_dataset(temperature_data, look_back)

# Split the data into training and testing sets
train_size = int(len(X) * 0.9)
train_X, test_X = X[:train_size], X[train_size:]
train_Y, test_Y = Y[:train_size], Y[train_size:]

# Reshape the input data to fit the LSTM model requirements [samples, time steps, features]
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))
test_X = np.reshape(test_X, (test_X.shape[0], test_X.shape[1], 1))

In [ ]:
train_X, test_X = merged_successive_train, merged_successive_test
train_Y, test_Y = output_train, output_test

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
# Build the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(2*INPUT_SIZE, 1)))
model.add(Dense(INPUT_SIZE))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(train_X, train_Y, epochs=100, batch_size=1, verbose=0)

# Evaluate the model
train_loss = model.evaluate(train_X, train_Y, verbose=0)
test_loss = model.evaluate(test_X, test_Y, verbose=0)
print(f"Train Loss: {train_loss:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Make predictions
train_predictions = model.predict(train_X)
predicted_data_lstm = model.predict(test_X)

In [ ]:
output_vals = dict()
lr_vals = dict()
mlp_vals = dict()
lstm_vals = dict()
pecnet_vals = dict()
pecnet_dwt_vals = dict()

In [ ]:
print("Window Size: %d" % INPUT_SIZE)
print("Data Type: T+RH->WS")
print("Metric: RMSE")
print("LR MAPE:", mean_squared_error(output_test, predicted_data_lr, squared=False))
print("MLP MAPE:", mean_squared_error(output_test, predicted_data_mlp, squared=False))
print("LSTM MAPE:", mean_squared_error(output_test, predicted_data_lstm, squared=False))
print("PECNET without wavelet MAPE:", mean_squared_error(output_test, predicted_data_pecnet_wout_dwt, squared=False))
print("PECNET with wavelet MAPE:", mean_squared_error(output_test, predicted_data_pecnet_dwt, squared=False))

In [ ]:
print("Window Size: %d" % INPUT_SIZE)
print("Data Type: T+T->T")
print("Metric: MAPE")
print("LR MAPE:", mean_absolute_percentage_error(output_test, predicted_data_lr))
print("MLP MAPE:", mean_absolute_percentage_error(output_test, predicted_data_mlp))
print("LSTM MAPE:", mean_absolute_percentage_error(output_test, predicted_data_lstm))
print("PECNET without wavelet MAPE:", mean_absolute_percentage_error(output_test, predicted_data_pecnet_wout_dwt))
print("PECNET with wavelet MAPE:", mean_absolute_percentage_error(output_test, predicted_data_pecnet_dwt))

In [ ]:
key = "8T+RH->WS"
output_vals[key] = output_test
lr_vals[key] = predicted_data_lr
mlp_vals[key] = predicted_data_mlp
lstm_vals[key] = predicted_data_lstm
pecnet_vals[key] = predicted_data_pecnet_wout_dwt
pecnet_dwt_vals[key] = predicted_data_pecnet_dwt

In [ ]:
key = "4T+RH->WS"

In [ ]:
plt.plot(range(len(output_vals[key][31:55])), [data[-1] for data in output_vals[key]][31:55], label='Actual')
plt.plot(range(len(output_vals[key][31:55])), [data[-1] for data in lr_vals[key]][31:55], label='LR')
plt.plot(range(len(output_vals[key][31:55])), [data[-1] for data in mlp_vals[key]][31:55], label='MLP')
plt.plot(range(len(output_vals[key][31:55])), [data[-1] for data in lstm_vals[key]][31:55], label='LSTM')
plt.plot(range(len(output_vals[key][31:55])), [data[-1] for data in pecnet_vals[key]][31:55], label='PECNET')
plt.plot(range(len(output_vals[key][31:55])), [data[-1] for data in pecnet_dwt_vals[key]][31:55], label='PECNET + DWT')
plt.xlabel('Time (hours)')

plt.ylabel('Wind Speed (m/s)')
#plt.ylabel('Temperature (°C)')
#plt.ylabel('Relative Humidity (%)')

plt.rc('font', size=15)          # controls default text sizes
plt.rc('axes', titlesize=8)     # fontsize of the axes title
plt.rc('axes', labelsize=8)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=15)    # fontsize of the tick labels
plt.rc('ytick', labelsize=15)    # fontsize of the tick labels
plt.rc('legend', fontsize=10)    # legend fontsize
plt.rc('figure', titlesize=15)  # fontsize of the figure title
plt.legend()
plt.show()

In [ ]:
temp = lr_vals.copy()

In [ ]:
for elm in temp:
    temp[elm] = temp[elm].tolist()

In [ ]:
output_vals[key] = output_test
lr_vals[key] = predicted_data_lr
mlp_vals[key] = predicted_data_mlp
lstm_vals[key] = predicted_data_lstm
pecnet_vals[key] = predicted_data_pecnet_wout_dwt
pecnet_dwt_vals[key] = predicted_data_pecnet_dwt

In [ ]:
import json

# Your dictionary
my_dict = temp

# File path where you want to save the dictionary data
file_path = "multi_station_lr_vals.json"

# Save the dictionary to the file
with open(file_path, "w") as file:
    json.dump(my_dict, file)

print("Dictionary saved to file successfully.")

In [ ]:
import json

# Specify the path to your JSON file
single_station_lr_vals = 'single_station_lr_vals.json'
single_station_mlp_vals = 'single_station_mlp_vals.json'
single_station_lstm_vals = 'single_station_lstm_vals.json'
single_station_output_vals = 'single_station_output_vals.json'
single_station_pecnet_dwt_vals = 'single_station_pecnet_dwt_vals.json'
single_station_pecnet_vals = 'single_station_pecnet_vals.json'

# Function to read JSON data from file and load into a dictionary
def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Call the function to read the JSON data into a dictionary
single_station_lr_vals = read_json_file(single_station_lr_vals)
single_station_mlp_vals = read_json_file(single_station_mlp_vals)
single_station_lstm_vals = read_json_file(single_station_lstm_vals)
single_station_output_vals = read_json_file(single_station_output_vals)
single_station_pecnet_dwt_vals = read_json_file(single_station_pecnet_dwt_vals)
single_station_pecnet_vals = read_json_file(single_station_pecnet_vals)

In [ ]:
key = '8WS'

In [ ]:
plt.plot(range(len(single_station_output_vals[key][21:45])), [data[-1] for data in single_station_output_vals[key]][21:45], label='Actual')
plt.plot(range(len(single_station_output_vals[key][21:45])), [data[-1] for data in single_station_lr_vals[key]][21:45], label='LR')
plt.plot(range(len(single_station_output_vals[key][21:45])), [data[-1] for data in single_station_mlp_vals[key]][21:45], label='MLP')
plt.plot(range(len(single_station_output_vals[key][21:45])), [data[-1] for data in single_station_lstm_vals[key]][21:45], label='LSTM')
plt.plot(range(len(single_station_output_vals[key][21:45])), [data[-1] for data in single_station_pecnet_vals[key]][21:45], label='PECNET')
plt.plot(range(len(single_station_output_vals[key][21:45])), [data[-1] for data in single_station_pecnet_dwt_vals[key]][21:45], label='PECNET + DWT')
plt.xlabel('Time (hours)')
plt.ylabel('Wind Speed (m/s)')
#plt.ylabel('Temperature (°C)')
#plt.ylabel('Relative Humidity (%)')

plt.rc('font', size=15)          # controls default text sizes
plt.rc('axes', titlesize=8)     # fontsize of the axes title
plt.rc('axes', labelsize=8)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=15)    # fontsize of the tick labels
plt.rc('ytick', labelsize=15)    # fontsize of the tick labels
plt.rc('legend', fontsize=10)    # legend fontsize
plt.rc('figure', titlesize=15)  # fontsize of the figure title
plt.legend()
plt.show()

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Read the data from the JSON file
file_path = 'tr-cities.json'  # Replace with the actual file path
data = gpd.read_file(file_path)

# Plot the map
fig, ax = plt.subplots(figsize=(10, 10))
data.plot(ax=ax, color='lightgrey', edgecolor='black')

# Mark the specified location on the map
target_location = gpd.GeoDataFrame({'geometry': gpd.points_from_xy([39.983722], [36.966])})
target_location.plot(ax=ax, color='red', markersize=100)
ax.axis('off')
ax.legend()
# Show the plot
plt.legend()
plt.show()


In [ ]:
import folium
import pandas as pd

# Read the CSV file
data = pd.read_csv('station_list.csv')

# Create a map centered around Turkey
map_turkey = folium.Map(location=[39.9208, 32.8541], zoom_start=6, tiles='cartodbpositron')

# Iterate over each row in the data and add a circle marker for each station
for index, row in data.iterrows():
    code = row['code']
    name = row['name']
    lat = row['lat']
    lon = row['lon']
    col = 'blue'
    if code == '63.09' or code == '63.10' or code == '63.14':
        continue
    if code == '63.11':
        col = 'green'
    if code == '63.12':
        col = 'red'
    
    # Add a circle marker to the map
    folium.CircleMarker(location=[lat, lon], radius=3, tooltip=f"{code}: {name}", fill=True, color=col, fill_color='blue').add_to(map_turkey)

# Save the map as an HTML file
map_turkey.save('turkey_map.html')


In [ ]:
for_print = get_data('original_code/data/hourly/63.11.HG.csv', type=5, start=12077, stop=13346)

dict_print = dict()
for i in range(200):
    if i < 125 or i >= 165:
        dict_print[i] = for_print[i]

In [ ]:
for_print = get_data('original_code/data/hourly/63.12.HG.csv', type=13, start=12077, stop=13346)

dict1 = dict()
dict2 = dict()
for i in range(200):
    if i < 83:
        dict1[i] = for_print[i]
    if i > 115:
        dict2[i] = for_print[i]

list1 = sorted(dict1.items()) # sorted by key, return a list of tuples
list2 = sorted(dict2.items()) # sorted by key, return a list of tuples
x1, y1 = zip(*list1) # unpack a list of pairs into two tuples
plt.plot(x1, y1, color='green')
x2, y2 = zip(*list2) # unpack a list of pairs into two tuples
plt.plot(x2, y2, color='green')

#plt.plot(range(200), dict_print)
plt.xlabel('Time (hours)')
plt.ylabel('Wind Speed (m/s)')
#plt.ylabel('Temperature (°C)')
#plt.ylabel('Relative Humidity (%)')

plt.rc('font', size=15)          # controls default text sizes
plt.rc('axes', titlesize=8)     # fontsize of the axes title
plt.rc('axes', labelsize=8)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=15)    # fontsize of the tick labels
plt.rc('ytick', labelsize=15)    # fontsize of the tick labels
plt.rc('legend', fontsize=10)    # legend fontsize
plt.rc('figure', titlesize=15)  # fontsize of the figure title
plt.show()